In [2]:
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from rpy2.robjects.packages import importr

utils = importr("utils")
utils.chooseCRANmirror(ind=1)
utils.install_packages('lme4')

%load_ext rpy2.ipython

R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/lme4_1.1-27.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 3311365 bytes (3.2 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to con

In [4]:
# Load data
conversations = pd.read_csv("results/contingency/conversations.csv")

# convert True/False to 0/1:
conversations.replace({False: 0, True: 1}, inplace=True)

conversations.head()


,utterance_id,speaker_code,transcript_raw,tokens,pos,start_time,end_time,age,corpus,transcript_file,...,follow_up_start_time,follow_up_is_speech_related,follow_up_is_intelligible,follow_up_is_contingent,follow_up_speech_act,response_latency,response_latency_follow_up,has_response,pos_feedback,response_is_clarification_request
0,9,CHI,&=vocalize .,['.'],['none'],15320.0,16139.0,12,Brent,/home/mitja/data/CHILDES/Brent/c1/000917.cha,...,19069.0,1.0,0,0,YY,-252.0,2930.0,1,1,0
1,157,CHI,&=vocalize .,['.'],['none'],367633.0,368084.0,12,Brent,/home/mitja/data/CHILDES/Brent/c1/000917.cha,...,370041.0,1.0,0,0,YY,528.0,1957.0,1,1,0
2,160,CHI,&=vocalize .,['.'],['none'],372007.0,372211.0,12,Brent,/home/mitja/data/CHILDES/Brent/c1/000917.cha,...,381680.0,1.0,0,0,YY,6830.0,9469.0,0,0,0
3,162,CHI,&=vocalize .,['.'],['none'],381680.0,381906.0,12,Brent,/home/mitja/data/CHILDES/Brent/c1/000917.cha,...,386103.0,1.0,0,0,YY,1965.0,4197.0,0,0,0
4,194,CHI,&=babble .,['.'],['none'],512973.0,513639.0,12,Brent,/home/mitja/data/CHILDES/Brent/c1/000917.cha,...,517483.0,1.0,0,0,YY,361.0,3844.0,1,1,0


In [5]:
# Some children have very few data points:
counts = conversations.groupby("child_name").size()
counts

child_name
Braunwald_Laura                7529
Brent_Alexander                 133
Brent_Allen                       3
Brent_Brooklyn                    8
Brent_Dillon                    532
Brent_Henry                      92
Brent_Jacob_Abernathy             4
Brent_Jaylen                     35
Brent_Maggie                    114
Brent_Miranda                   105
Brent_Morgan                    414
Brent_Tabitha                   187
Brent_Tabitha_Sims                3
Brent_Timothy                   306
Brent_Tyrese                      6
Brent_Vas                        11
Brent_Vas_Coleman                87
Brent_Xavier                      9
MPI-EVA-Manchester_Eleanor    40925
MPI-EVA-Manchester_Fraser     90496
Thomas_Brian                    302
Thomas_Thomas                 90240
dtype: int64

In [11]:
# Exclude children with less than 100 datapoints:
# child_names_enough_data = [name for name, count in counts.items() if count > 100]
# print(len(conversations))
# conversations = conversations[conversations.child_name.isin(child_names_enough_data)]
# print(len(conversations))

In [5]:
# normalize age
min_age, max_age = conversations["age"].min(), conversations["age"].max()
conversations["age"] = (conversations["age"] - min_age) / (max_age - min_age) * (1 - 0)

## Quality of communicative feedback/ Caregiver contingency


### Timing:

In [14]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('has_response ~ is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: has_response ~ is_intelligible * age_normalized + (1 | child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
173345.9 173397.7 -86668.0 173335.9   231536 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4872  0.3109  0.3410  0.3895  0.9238 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 0.2714   0.5209  
Number of obs: 231541, groups:  child_name, 22

Fixed effects:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     0.78918    0.10351   7.624 2.46e-14 ***
is_intelligible                 0.79457    0.04085  19.449  < 2e-16 ***
age_normalized                  0.66146    0.06743   9.810  < 2e-16 ***
is_intelligible:age_normalized  0.44585    0.07488   5.954 2.62e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.

### Clarification requests

In [17]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('response_is_clarification_request ~ is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
response_is_clarification_request ~ is_intelligible * age_normalized +  
    (1 | child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
 15518.4  15570.2  -7754.2  15508.4   231536 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-0.283 -0.075 -0.072 -0.031 36.393 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 1.056    1.028   
Number of obs: 231541, groups:  child_name, 22

Fixed effects:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     -3.7426     0.2366 -15.819  < 2e-16 ***
is_intelligible                 -2.6123     0.1223 -21.367  < 2e-16 ***
age_normalized                  -2.0319     0.2035  -9.987  < 2e-16 ***
is_intelligible:age_normalized   1.6127     0.2326   6.934 4.09e-12 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.

### Combined (Positive Feedback = No pause, no clarification request)

In [18]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('pos_feedback ~ is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: pos_feedback ~ is_intelligible * age_normalized + (1 | child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
177320.8 177372.6 -88655.4 177310.8   231536 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.2027  0.3113  0.3407  0.4003  0.9944 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 0.3022   0.5497  
Number of obs: 231541, groups:  child_name, 22

Fixed effects:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     0.62859    0.09246   6.798 1.06e-11 ***
is_intelligible                 1.00836    0.04093  24.636  < 2e-16 ***
age_normalized                  0.89248    0.06463  13.809  < 2e-16 ***
is_intelligible:age_normalized  0.19024    0.07467   2.548   0.0108 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.

## Effect of communicative feedback
### Positive Feedback: Timing

In [7]:
%%R -i conversations
library(lme4)

conversations_child_intelligible = subset(conversations, is_intelligible==1)
# TODO: only intelligible ones?

m_child_contingency<-glmer('follow_up_is_intelligible ~ has_response * age + (1 | child_name)', data=conversations_child_intelligible, family=binomial)
print(summary(m_child_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: follow_up_is_intelligible ~ has_response * age + (1 | child_name)
   Data: conversations_child_intelligible

     AIC      BIC   logLik deviance df.resid 
125671.7 125722.9 -62830.8 125661.7   210249 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4316  0.2664  0.2895  0.3307  1.3472 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 1.181    1.087   
Number of obs: 210254, groups:  child_name, 21

Fixed effects:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)       1.54459    0.12530  12.327  < 2e-16 ***
has_response      0.32964    0.04483   7.354 1.93e-13 ***
age               1.09402    0.07180  15.237  < 2e-16 ***
has_response:age -0.09705    0.07939  -1.222    0.222    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (

### Negative Feedback: Clarification requests

In [11]:
%%R -i conversations
library(lme4)

conversations_unintelligible = subset(conversations, is_intelligible == 0)


m_child_contingency<-glmer('follow_up_is_intelligible ~ response_is_clarification_request * age + (1 | child_name)', data=conversations_unintelligible, family=binomial)
print(summary(m_child_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: follow_up_is_intelligible ~ response_is_clarification_request *  
    age + (1 | child_name)
   Data: conversations_unintelligible

     AIC      BIC   logLik deviance df.resid 
 34111.7  34153.1 -17050.9  34101.7    29291 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.3318 -1.1835  0.6085  0.6968 11.1542 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 5.405    2.325   
Number of obs: 29296, groups:  child_name, 17

Fixed effects:
                                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)                           -1.64034    0.47541  -3.450  0.00056 ***
response_is_clarification_request     -0.59214    0.23912  -2.476  0.01328 *  
age                                    1.54679    0.08572  18.045  < 2e-16 ***
response_is_clarification_request:age  1.23226    0.51790   2.3